In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import pipeline
import math
import gradio as gr

# Создаем датасет из своих данных

В качестве датасета я использовал несколько наборов "пацанских" цитат, спарщенных с соответствующих групп во вконтакте и сайтов в интернете.

In [11]:
data = []

with open('datasets/serious.txt', encoding='utf-8') as my_file:
    for line in my_file:
        data.append(line[:-1])

with open('datasets/vanilla.txt', encoding='utf-8') as my_file:
    for line in my_file:
        data.append(line[:-1])

In [12]:
with open('datasets/volk.cleaned.big.txt', encoding='utf-8') as my_file:
    for line in my_file:
        for i in line.split('^'):
            i.replace('\n', ' ')
            data.append(i)

In [25]:
print (f'"{data[228]}" - пример цитаты из датасета')

"тебя ценят только тогда когда в тебе нуждаются" - пример цитаты из датасета


In [26]:
train, val = train_test_split(data, test_size=0.2, random_state=13)

print (f'{len(train)} - количество цитат в тренировчном датасете')
print (f'{len(val)} - количество цитат в тестовом датасете')

train = ". ".join(train)
val = ". ".join(val)

print (f'{len(train)} - количество символов в тренировчном датасете')
print (f'{len(val)} - количество символов в тестовом датасете')

with open('datasets/train.txt', 'w', encoding='utf-8') as f:
    f.write(train)

with open('datasets/val.txt', 'w', encoding='utf-8') as f:
    f.write(val)

3046528 - количество цитат в тренировчном датасете
761633 - количество цитат в тестовом датасете
158246699 - количество символов в тренировчном датасете
39590699 - количество символов в тестовом датасете


In [27]:
del data, train, val

In [28]:
train_path = 'datasets/train.txt'
test_path = 'datasets/val.txt'

# Импортируем предобученную модель

В качестве базовой модели было решено использовать Сберовскую rugpt3medium

In [9]:
text_generator = pipeline('text-generation', model="ai-forever/rugpt3medium_based_on_gpt2")

In [10]:
model = text_generator.model
tokenizer = text_generator.tokenizer

# Токенизируем данные и создаем датасет для обучения

In [3]:
from transformers import TextDataset,DataCollatorForLanguageModeling

In [12]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset, data_collator = load_dataset (train_path,test_path,tokenizer)

D:\ProjectsPy\clean_giga_generator\venv\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


# Файнтюним модель

In [19]:
training_args = TrainingArguments(
    output_dir="./ГигаЦитаты",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32, 
    eval_steps = 400, 
    save_steps=800,
    warmup_steps=500,
    prediction_loss_only=True,
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [15]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=150, training_loss=3.960201416015625, metrics={'train_runtime': 4074.0953, 'train_samples_per_second': 0.579, 'train_steps_per_second': 0.037, 'total_flos': 547933409771520.0, 'train_loss': 3.960201416015625, 'epoch': 10.0})

# Оценка модели

Оценка текстово пораждающих моделей - весьма трудная задача. Но давайте хотя бы посмотрим на Perplexity

In [101]:

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 29.35


Это просто достаточно хорошее значение для модели - сравнивать её с другими моделями достаточно глупо, так как Perplexity строго привязана к словарю модели. Но все же это значение сопоставимо с "бенчмарком" в виде значений умных моделей на умных датасетах 

(например WikiText-103:  https://paperswithcode.com/sota/language-modelling-on-wikitext-103) 

# Сохранение модели

In [ ]:
trainer.save_model()
tokenizer.save_pretrained("./tokenizer")

# Создадим удобный интерфейс для взаимодействия с моделью

In [29]:
text_gen = pipeline('text-generation', model = './ГигаЦитаты', tokenizer="./tokenizer")

def generate(text, temperature = 1.0, max_length = 20, number_of_samples = 5):
    
    temperature = float(temperature)
    max_length = int(max_length)
    number_of_samples = int(number_of_samples)
    results = text_gen(text, num_return_sequences=number_of_samples, temperature = temperature, return_full_text=True, min_length = 10, max_length=max_length, do_sample=True)
    texts = [i['generated_text'] for i in results]
    texts_clean = []
    
    for number, i in enumerate(texts):

        text_with_num = f'{number+1}) ' + i
        texts_splitted = text_with_num.split('.')
        
        if len(texts_splitted)>=2:
            texts_clean.append ('.'.join(texts_splitted[:-1]))
        else:
            texts_clean.append ('.'.join(texts_splitted))

    return "\n\n".join(texts_clean)

In [30]:
with gr.Blocks() as demo:
    
    gr.Markdown(
    """
    # НЕЙРОСТЕТХЕМ
    
    Настройте свои параметры генерации настоящих поцанских цитат с легким оттенком ванили.
    
    """)
    
    temperature = gr.Textbox(label="Температура модели (рекомендуемое: 1.0)", value = "1.0", lines=1)
    max_len = gr.Textbox(label="Максимальная длина", value = "20", lines=1)
    text = gr.Textbox(label="Входной текст", value = "Настоящий пацан, это тот, кто", lines=1)
    number_of_samples = gr.Textbox(label="Количество вариантов", value = "5", lines=1)
    text_out = gr.Textbox(label="МУДРОСТЬ ОТ МОДЕЛИ", lines=10)
    
    btn = gr.Button(value="ДАЙ МУДРОСТЬ")
    btn.click(generate, inputs=[text, temperature, max_len, number_of_samples], outputs=[text_out])

In [31]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://52d431c47ba289ad01.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# А ещё мы это все и захостим (потыкаться в модель можно по ссылке - все работает)

## https://huggingface.co/spaces/vivatimperial/NeuralStatham

Бесплатный хостинг через spaces - великая вещь